### Part8 - Deep Learning - Convolutional Neural Networks(CNN)

### Lecture note

- reference : http://taewan.kim/post/cnn/

#### image

- image를 숫자로 표현하고 색을 구하고 표현


#### convolution

- convolution is a combined integration of the two functions and it shows you how one function modifies the other or modifies the shape
- 자세한건 이미지 참고, 논문 참고

- filter?
  - feature detector를 이용하여 input image를 순회하여(**stride**) feature map을 완성
  - 완성된 map에서 가장 높은 값을 지니는 부분을 확인한다.
  
- we create many feature maps to obtain our first convolution layer
- filter를 이용하면 다양한 이미지 결과를 이끌어 낼 수 있다.(이미지 참고)

#### ReLU layer

- Rectifier
  - image are highly non-linear
  - 제시하는 논문 읽어 볼 것


#### Pooling

max pooling
- feature map에서 특정 array내에서 가장 큰 값을 뽑아서 pooled feature map을 만든다.
- reduce size, reduce number of parameter, preventing overfitting
- research paper 보세요. 매우 간단하고 쉬워요. 꼭 보세요. ~10페이지임~

#### Flattening

pooled Feature Map에서 시작할거임
- we flatten everything into a long vector
- 1차원으로 만든다고 생각하면 편함

#### Full connection

- dog이 될 수 있는 뉴런에 대해 반복적으로 러닝이 진행되면서 결과를 도출한다.
- cat도 동일한 원리로 작동된다.
- 결국 확률적으로 선택하는 원리이다.


#### summary

- 이미지 넣기
- 논문 넣기

#### Softmax & Cross_entropy

Softmax

- soft max fuction or the mormalized exponential function is a generalization of the logistic function
- [ref](https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221021710286&proxyReferer=https%3A%2F%2Fwww.google.com%2F)
- softmax를 이용(지수 함수)하면서 크고 작음의 차이가 더 벌어지게 되고, 분류 문제에서 더 올바른 결과를 이끌어 낼 수 있는 학습이 가능하다.

Cross_entropy

- [ref](http://blog.naver.com/PostView.nhn?blogId=gyrbsdl18&logNo=221013188633&redirect=Dlog&widgetTypeCall=true)
- [ref](https://curt-park.github.io/2018-09-19/loss-cross-entropy)
- 왜 이걸 쓰는게 mean squared error 쓰는 것보다 낫냐?
  - 매우 작은 숫자들을 사용하게 되는데 그럴 때 매우 작은 수들로 연산을 하면 변화가 없음.
  - cross_entropy를 사용하면 log scale을 사용하기 때문에 좀 더 연산에 최적화를 시킬 수 있다.

### Code!!

In [1]:
# Part 1 - Building the CNN

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [2]:
# initialising the CNN

classifier = Sequential()

In [4]:
# step 1 - Convolution

classifier.add(Convolution2D(32, 3, 3, 
                             border_mode = 'same', 
                             input_shape = (64, 64, 3), # CNN 할 때 이렇게 넣어줘야 함
                             activation = 'relu'
                            )
              )

/Users/wontaek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu", padding="same")`
  


In [5]:
# step 2 - Pooling

classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# adding a second convolutional layer
# only need dimension, activation on second layer


classifier.add(Convolution2D(32, 3, 3, 
                             border_mode = 'same', 
                             activation = 'relu'
                            )
              )
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [7]:
# step 3 - Flattening

classifier.add(Flatten())

In [9]:
# step 4 - Full connection

classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

/Users/wontaek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/wontaek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  after removing the cwd from sys.path.


In [10]:
# compiling the CNN

classifier.compile(optimizer='adam', 
                   loss='binary_crossentropy', 
                   metrics = ['accuracy']
                  )

### 기억하자

- keras document에 방문하면 기본적인 코드가 있다. 그걸 가져다 쓰면 된다.
  - https://keras.io/preprocessing/image/#imagedatagenerator-class

In [17]:
# part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
from scipy import ndimage


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

/Users/wontaek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/wontaek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64), # cnn model target size
    batch_size=32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


Found 8001 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [20]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000/32,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000/32)



# classifier.fit_generator(
#         training_set,
#         steps_per_epoch=8000,
#         epochs=25,
#         validation_data=test_set,
#         validation_steps=2000)

Epoch 1/25
250/250 [==============================] - 84s 337ms/step - loss: 0.5580 - acc: 0.7132 - val_loss: 0.6099 - val_acc: 0.6645
Epoch 2/25
250/250 [==============================] - 83s 333ms/step - loss: 0.5210 - acc: 0.7426 - val_loss: 0.5112 - val_acc: 0.7555
Epoch 3/25
250/250 [==============================] - 83s 333ms/step - loss: 0.4954 - acc: 0.7528 - val_loss: 0.5440 - val_acc: 0.7360
Epoch 4/25
250/250 [==============================] - 83s 333ms/step - loss: 0.4770 - acc: 0.7792 - val_loss: 0.4970 - val_acc: 0.7640
Epoch 5/25
250/250 [==============================] - 86s 343ms/step - loss: 0.4562 - acc: 0.7819 - val_loss: 0.4732 - val_acc: 0.7735
Epoch 6/25
250/250 [==============================] - 83s 334ms/step - loss: 0.4546 - acc: 0.7825 - val_loss: 0.4932 - val_acc: 0.7765
Epoch 7/25
250/250 [==============================] - 84s 337ms/step - loss: 0.4263 - acc: 0.8000 - val_loss: 0.5229 - val_acc: 0.7695
Epoch 8/25
250/250 [==============================] - 8